-  Importing Libraries

In [ ]:
import os  
import time
import numpy as np    #work with numpy arrays
import pandas as pd    #working with dataframes
import seaborn as sns  #rendering beautiful plots
import matplotlib.pyplot as plt #python's basic plotting library
from sklearn.preprocessing import LabelEncoder    #encoding categories to numerical values
#to view plots inline here's a magic function
%matplotlib inline  
import warnings     #to remove any warnings that causes headache
warnings.filterwarnings('ignore')

-  Reading csv files provided.
    -  X_train.csv 
    - X_test.csv
    -  sample_submission.csv
    -  y_train.csv

In [ ]:
train=pd.read_csv('../input/X_train.csv')
y=pd.read_csv('../input/y_train.csv')
test=pd.read_csv('../input/X_test.csv')
sub = pd.read_csv('../input/sample_submission.csv')
print("\nX_train shape: {}, X_test shape: {}".format(train.shape, test.shape))
print("y_train shape: {}, submission shape: {}".format(y.shape, sub.shape))

**display_all() function**
-  It will allow to print specified no. of rows and columns mentioned in the function.

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows",4,"display.max_columns",10):
        display(df)
display_all(train)

# This is a classification problem with nine possible classes (floor surfaces):
-  
['fine_concrete',
 'concrete',
 'soft_tiles',
 'tiled',
 'soft_pvc',
 'hard_tiles_large_space',
 'carpet',
 'hard_tiles',
 'wood']

In [ ]:
list(pd.unique(y['surface']))

**Plotting Count Plot**
-  It can be observed that there's a certain imbalance in the categories, that can be handled using Upsampling ( But will not be done in this notebook )

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Training labels")
ax = sns.countplot(y='surface', data=y)

# Feature Engineering
-  Creating new features from the one's already there.

In [ ]:
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

def feature_extraction(actual):
    new = pd.DataFrame()
    actual['total_angular_velocity'] = actual['angular_velocity_X'] + actual['angular_velocity_Y'] + actual['angular_velocity_Z']
    actual['total_linear_acceleration'] = actual['linear_acceleration_X'] + actual['linear_acceleration_Y'] + actual['linear_acceleration_Z']
    
    actual['acc_vs_vel'] = actual['total_linear_acceleration'] / actual['total_angular_velocity']
    
    x, y, z, w = actual['orientation_X'].tolist(), actual['orientation_Y'].tolist(), actual['orientation_Z'].tolist(), actual['orientation_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    actual['euler_x'] = nx
    actual['euler_y'] = ny
    actual['euler_z'] = nz
    
    def f1(x):
        return np.mean(np.diff(np.abs(np.diff(x))))
    
    def f2(x):
        return np.mean(np.abs(np.diff(x)))
    
    for col in actual.columns:
        if col in ['row_id', 'series_id', 'measurement_number']:
            continue
        new[col + '_mean'] = actual.groupby(['series_id'])[col].mean()
        new[col + '_min'] = actual.groupby(['series_id'])[col].min()
        new[col + '_max'] = actual.groupby(['series_id'])[col].max()
        new[col + '_std'] = actual.groupby(['series_id'])[col].std()
        new[col + '_max_to_min'] = new[col + '_max'] / new[col + '_min']
        
        # Change. 1st order.
        new[col + '_mean_abs_change'] = actual.groupby('series_id')[col].apply(f2)
        
        # Change of Change. 2nd order.
        new[col + '_mean_change_of_abs_change'] = actual.groupby('series_id')[col].apply(f1)
        
        new[col + '_abs_max'] = actual.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
        new[col + '_abs_min'] = actual.groupby('series_id')[col].apply(lambda x: np.min(np.abs(x)))

    return new

**Writing a function that will present the missing values in a table well presented**

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        
        return mis_val_table_ren_columns
     

In [ ]:
train_df = feature_extraction(train)
test_df = feature_extraction(test)
# train_df.to_csv('New_train2.csv')
# test_df.to_csv('New_test2.csv')

**Checking Missing values for train_df**
-  It may have been there because of the use of tan function in the code which turns 0 to infinity. Just to be sure this check has been made.

In [ ]:
missing_values_table(train_df) 

**It can be seen that there are two columns having 2-2 missing values each**
-  They must be removed

In [ ]:
train_df['acc_vs_vel_std']=train_df['acc_vs_vel_std'].fillna(0)
train_df['acc_vs_vel_mean_change_of_abs_change']=train_df['acc_vs_vel_mean_change_of_abs_change'].fillna(0)

In [ ]:
missing_values_table(train_df) 

In [ ]:
train_df.fillna(0, inplace = True)
test_df.fillna(0, inplace = True)
train_df.replace(-np.inf, 0, inplace = True)
train_df.replace(np.inf, 0, inplace = True)
test_df.replace(-np.inf, 0, inplace = True)
test_df.replace(np.inf, 0, inplace = True)

In [ ]:
train_df=train_df.astype('float32')
test_df=test_df.astype('float32')
train_df.info()

**Label Encoding**
- Encoding categories into numerical values.

In [ ]:
le = LabelEncoder()
target = le.fit_transform(y['surface'])    #label Encoding is required to convert names to num

In [ ]:
target

# Trying RandomForestClassifier from sklearn.ensemble


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(random_state=42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

# Random Forest HyperParameter tuning 
-  Gathering more data and feature engineering usually has the greatest payoff in terms of time invested versus improved performance, but when we have exhausted all data sources, it’s time to move on to model hyperparameter tuning.

### Provided above are some of the hyperparameters we use in random forest.
- That is quite an overwhelming list! How do we know where to start? A good place is the documentation on the random forest in Scikit-Learn. This tells us the most important settings are the number of trees in the forest (n_estimators) and the number of features considered for splitting at each leaf node (max_features). We could go read the research papers on the random forest and try to theorize the best hyperparameters, but a more efficient use of our time is just to try out a wide range of values and see what works! We will try adjusting the following set of hyperparameters:
    -  n_estimators = number of trees in the foreset
    - max_features = max number of features considered for splitting a node
    - max_depth = max number of levels in each decision tree
    - min_samples_split = min number of data points placed in a node before the node is split
    - min_samples_leaf = min number of data points allowed in a leaf node
    - bootstrap = method for sampling data points (with or without replacement)
    
## Random Hyperparamter Grid    
- Usually, we only have a vague idea of the best hyperparameters and thus the best approach to narrow our search is to evaluate a wide range of values for each hyperparameter. Using Scikit-Learn’s RandomizedSearchCV method, we can define a grid of hyperparameter ranges, and randomly sample from the grid, performing K-Fold CV with each combination of values.
-  To use RandomizedSearchCV, we first need to create a parameter grid to sample from during fitting:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

# Uncomment and run below cells to train your own model.

On each iteration, the algorithm will choose a difference combination of the features. Altogether, there are 2 * 12 * 2 * 3 * 3 * 10 = 4320 settings! However, the benefit of a random search is that we are not trying every combination, but selecting at random to sample a wide range of values.

**Run Below Code to run RandomizedSearchCV**

In [ ]:
# rf = RandomForestClassifier()

# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

# rf_random.fit(train_df, target)

**The Best parameter that i got from above RandomSearch are :**
 - 'bootstrap': False
 - 'max_depth': 50
 - 'max_features': 'auto'
 - 'min_samples_leaf': 1
 - 'min_samples_split': 2
 - 'n_estimators': 1000

**Now we will extract the best estimator and will make our test data run on this model.**

In [ ]:
# best_random = rf_random.best_estimator_
# best_random.get_params()
# pred=best_random.predict(test_df)

In [ ]:
# sub['surface'] = le.inverse_transform(pred)
# sub.to_csv('sample2.csv', index=False)

**Submitting sample2.csv scored 0.69  on the leaderboard**

# 2nd attempt

# Create the parameter grid based on the results of random search 
-  Random search allowed us to narrow down the range for each hyperparameter. Now that we know where to concentrate our search, we can explicitly specify every combination of settings to try. We do this with GridSearchCV, a method that, instead of sampling randomly from a distribution, evaluates all combinations we define. To use Grid Search, we make another grid based on the best values provided by random search:


In [ ]:
# from sklearn.model_selection import GridSearchCV
# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [False],
#     'max_depth': [40, 50, 60, 80],
#     'max_features': ['auto','sqrt'],
#     'min_samples_leaf': [1, 2, 3],
#     'min_samples_split': [4, 3, 2],
#     'n_estimators': [500, 900, 1000, 1200]
# }
# # Create a based model
# rf = RandomForestClassifier()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1, verbose = 3)
# # Fit the grid search to the data
# grid_search.fit(train_df, target)

**The following hyperparameters were obtained from more exhaustive GridSearchCV**
-  max_depth: 80
-  max_features: 'auto'
-  n_estimators: 900
-  min_samples_split : 3
-  min_samples_leaf : 1
- bootstrap: False

In [ ]:
# best_model=grid_search.best_estimator_
# x=grid_search.best_params_
# best_model.set_params()
# best_model.get_params()

**Retraining our best estimator on the training data and finding the predictions**

In [ ]:
# best_model.fit(train_df,target)

In [ ]:
# pred2=best_model.predict(test_df)
# pred2

In [ ]:
# sub['surface'] = le.inverse_transform(pred2)
# sub.to_csv('sample-4.csv', index=False)
# sub.head(3)

**The Above submission scored 0.70 on the leaderboard.**
